In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.8 MB/s eta 0:00:00


In [ ]:
!pip install datasets rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=6c7d85b38883d64605fda7916b8868b51404db375f627db1453bb12785a37dc5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModel
from transformers import AutoTokenizer, BartForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
# model = AutoModel.from_pretrained("facebook/bart-base")

In [ ]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub and get a sample
dataset = load_dataset('cnn_dailymail','3.0.0')
# sample = dataset['test'][randrange(len(dataset["test"]))]
test_set = dataset['test']
sample = dataset['test'].select(range(10))

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
test_set

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})

## Different Approach (works) (Uses CPU, takes too long)

In [ ]:
from re import VERBOSE

def generate_summary(batch):
    # Ensure padding and truncation
    inputs = tokenizer(batch["article"], return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=120,
        do_sample=False,
        #top_p=0.9
    )
    # Decode all generated summaries
    summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return {"summary": summaries}

# -----------------------

# Use the map function to apply the generate_summary function to each sample in the dataset  <--- just for sample of 10
# summary_dataset = dataset.map(generate_summary, batched=True, batch_size=10)

# Process the dataset with progress bar & increase batch_size for processing full test dataset
summary_dataset = test_set.map(generate_summary, batched=True, batch_size=16)



# Extract generated and reference summaries
generated_summaries = summary_dataset["summary"]
reference_summaries = [item["highlights"] for item in test_set]

# -----------------------

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

In [ ]:
# Now compute ROUGE scores
scores = metric.compute(predictions=generated_summaries, references=reference_summaries)

# Output the scores
print(scores)

In [ ]:
#

## Try using GPU instead (works better)

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")


Using Tesla T4


In [ ]:
model = model.to(device)


In [ ]:
def generate_summary(batch):
    # Ensure padding and truncation
    inputs = tokenizer(batch["article"], return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU

    # Generate summaries
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=120,
        do_sample=False,
        #top_p=0.9
    )
    outputs = outputs.cpu()  # Move outputs to CPU

    # Decode all generated summaries
    summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return {"summary": summaries}


In [ ]:
# Process the dataset with progress bar & increase batch_size for processing full test dataset
summary_dataset = test_set.map(generate_summary, batched=True, batch_size=16)



# Extract generated and reference summaries
generated_summaries = summary_dataset["summary"]
reference_summaries = [item["highlights"] for item in test_set]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

<ipython-input-14-faaac16489c9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
# Now compute ROUGE scores
scores = metric.compute(predictions=generated_summaries, references=reference_summaries)

# Output the scores
print(scores)

{'rouge1': AggregateScore(low=Score(precision=0.2968356851546808, recall=0.5588132704722116, fmeasure=0.3774173988493247), mid=Score(precision=0.2988914888123473, recall=0.5614585673755041, fmeasure=0.3794214398432759), high=Score(precision=0.3008698487518703, recall=0.5643883889758657, fmeasure=0.381404097068721)), 'rouge2': AggregateScore(low=Score(precision=0.1320433015156744, recall=0.25081451803956245, fmeasure=0.16813368832397035), mid=Score(precision=0.13361278863868536, recall=0.2535954415463456, fmeasure=0.17006449190375786), high=Score(precision=0.1353746950490639, recall=0.25654781230953216, fmeasure=0.17210413350192547)), 'rougeL': AggregateScore(low=Score(precision=0.18545686871875455, recall=0.3553928891150033, fmeasure=0.2372642567439469), mid=Score(precision=0.1869929685770555, recall=0.3580025659013452, fmeasure=0.23892518691018222), high=Score(precision=0.18839345398577692, recall=0.36054866055088236, fmeasure=0.24058197478183233)), 'rougeLsum': AggregateScore(low=Sco

Results on full test set

'rouge1':

low=Score(precision=0.2968356851546808, recall=0.5588132704722116, fmeasure=0.3774173988493247), mid=Score(precision=0.2988914888123473, recall=0.5614585673755041, fmeasure=0.3794214398432759), high=Score(precision=0.3008698487518703, recall=0.5643883889758657, fmeasure=0.381404097068721)),

'rouge2':

low=Score(precision=0.1320433015156744, recall=0.25081451803956245, fmeasure=0.16813368832397035), mid=Score(precision=0.13361278863868536, recall=0.2535954415463456, fmeasure=0.17006449190375786), high=Score(precision=0.1353746950490639, recall=0.25654781230953216, fmeasure=0.17210413350192547)),

'rougeL':

low=Score(precision=0.18545686871875455, recall=0.3553928891150033, fmeasure=0.2372642567439469), mid=Score(precision=0.1869929685770555, recall=0.3580025659013452, fmeasure=0.23892518691018222), high=Score(precision=0.18839345398577692, recall=0.36054866055088236, fmeasure=0.24058197478183233)),

'rougeLsum':

low=Score(precision=0.24366947269260972, recall=0.45918456046531836, fmeasure=0.3098571350434835), mid=Score(precision=0.2454750580139858, recall=0.46170910416010186, fmeasure=0.31177569144121586), high=Score(precision=0.24721233769303935, recall=0.46444144435846607, fmeasure=0.3135459223860757))}

Fine-tune

0.2503
0.1253
0.2087
